##### Imports

In [1]:
import os
import time
import ee
import requests
import shutil, sys
import logging
import multiprocessing
from retry import retry

### Earth Engine Authentication

In [ ]:
ee.Authenticate()

In [3]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

### Data Loading

In [ ]:
def mask_L8_SR (img):
    '''takes in an image and sets a mask for its low quality pixels'''
    qa = img.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3)
    shadow = qa.bitwiseAnd(1 << 4)
    Mask = (cloud.neq(0)).Or(shadow.neq(0)).uint8()
    return  img.updateMask(Mask.eq(0))


#### Year selection

In [ ]:
# ---> ENTER Year BELOW <---
YEAR = 2021
print(f'YEAR: {YEAR}')

#### Country selection

In [ ]:
# ---> ENTER COUNTRY BELOW <---
COUNTRY = "Democratic Republic of the Congo"

countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
border = countries.filter(ee.Filter.eq('ADM0_NAME', COUNTRY))
img = ee.Image(1).clip(border)

pointsROI = img.stratifiedSample(numPoints = 1000000,
                                classBand = 'constant',
                                region = border,
                                scale = 10000,
                                geometries = True)

print(f'COUNTRY: {COUNTRY}, POINTS: {pointsROI.size().getInfo()}')

#### Datasets

In [ ]:
# Landsat-8 (input)
L8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                .filterDate(f'{YEAR}-01-01',f'{YEAR + 1}-01-01')
                .map(lambda x: mask_L8_SR(x)
                .select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
                        ,['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']))).median() 


In [ ]:
# Nightlight (input)
VIIRS = (ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')
                  .filterDate(f'{YEAR}-01-01',f'{YEAR + 1}-01-01')
                  .select(['avg_rad'],['NL'])).median()


In [ ]:
# Esa WorldCover 2020 and ESRI LULC(target)
esa_wc2020 = ee.ImageCollection("ESA/WorldCover/v100").mosaic()
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS").filterDate('2020-01-01','2020-12-31').mosaic()


In [ ]:
# GHS_SMOD
smod_asset = ee.Image('projects/ee-albinso/assets/GHS_SMOD_2020_GLOBE_REPROJECTED') #.reproject('epsg:4326', None, 1000)

#### Data retrieval

In [ ]:
# specify output directory
OUT_DIR = f'/mimer/NOBACKUP/groups/globalpoverty1/albin_and_albin/raw_data_newest/{YEAR}/DemocraticRepublicoftheCongo'

# create directory if it does not exist
if not os.path.isdir(OUT_DIR):
        os.makedirs(OUT_DIR, 0o775)
        print(f'Directory created (mimer): ..{OUT_DIR.partition("globalpoverty1")[-1]}')

In [ ]:
@retry(tries=10, delay=1, backoff=2)
def get_image_tile(point):
    '''Retrieve image tile at given point, scale and dimension.  Write to directory'''
    
    # get ID and point-coordinates
    ID = point['id']
    point = ee.Geometry.Point(point['geometry']['coordinates'])
    
    # set up rectangular bound around point
    ROI = point.buffer(500*10).bounds()  # tile dim: 1000*1000px (1px=10m)
    
    # images to retrieve
    imgLandsat = L8.clip(ROI)
    imgNL = VIIRS.clip(ROI)
    imgTarget_esa = esa_wc2020.clip(ROI)
    imgTarget_esri = esri_lulc10.clip(ROI)
    imgSMOD = smod_asset.clip(ROI)#.reproject('epsg:4326', None, 1000)
    
    # concatenate input
    #imageInput = ee.Image.cat([imgLandsat, imgNL, imgSMOD, imgTarget_esa, imgTarget_esri])
    imageInput = ee.Image.cat([imgLandsat, imgNL, imgTarget_esri])

    # fetch the URL from which to download the image.
    url = imageInput.float().getDownloadUrl({
        'scale': 10,
        'dimensions': '1000x1000',
        'format': 'GEO_TIFF'
    })
    r = requests.get(url) # send get request
    
    # save retrieved tile
    if r.status_code == 200:  # HTTP GET: 200 OK
        filename = OUT_DIR + f'/tile_{ID}.tif_2021'
        with open(filename, 'wb') as out_file:
              out_file.write(r.content)
    # retry, get request failed
    else:
        #print(f'{r.status_code}: {r.reason}')
        raise HTTPException(status_code=r.status_code, detail=r.reason)
    
    return r.content


In [ ]:
# make a list of the points
image_points = pointsROI.toList(pointsROI.size().getInfo()).getInfo()

point_to_download = image_points[15717]
print(point_to_download)

# get corresponding image tiles
get_image_tile(point_to_download)

print(f'redownload complete: {COUNTRY}, tile_{point_to_download["id"]}')

In [ ]:
print(point_to_download["id"])